In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# =============================================================================
# ** == STILL TO DO
# =============================================================================
# Timeseries variables 
# start time = can get  fromn ini, or leave blank?
# rate = 
# =============================================================================
# Script to build NWB Files
# =============================================================================

# Python Imports
from datetime import datetime
import h5py
import numpy as np
import glob
import os
import pandas as pd
from openpyxl import Workbook, load_workbook

# matlab Imports
import matlab.engine
eng = matlab.engine.start_matlab()

# Reading the Experiment Data sheet
from docx import Document

# NWB imports
from pynwb import NWBFile, get_manager
from pynwb import TimeSeries, behavior
from pynwb import NWBHDF5IO

# my scripts
#import from_ini # this has crashed for some reason????? 

/Users/solomia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def sbj_sheet_list(datas_dir):
    data_sheet = pd.ExcelFile(datas_dir)
    sheets = data_sheet.sheet_names
    return(sheets)

def get_exp_data(data_sheet):
    exp_details = pd.read_excel(data_sheet, sheet_name= 'Experiment Details', index_col=0)
    return(exp_details)

def create_mouse_id_folder(mouse_id):
    
    """ Creates folder with Mouse ID """

    target_dir = os.path.join('/Users/solomia/Dropbox/2p_da/data_files/nwb_folder/', mouse_id)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    return target_dir

def create_mouse_date_folder(mouse_id, exp_date):
    mouse_target_dir = create_mouse_id_folder(mouse_id)
    target_dir = os.path.join(mouse_target_dir, exp_date)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    return target_dir

def is_sbj_sheet(sheet):
    if 'template' in sheet.lower():
        return False
    if sheet.lower().startswith('subject details'):
        return True
    return False

def twop_TS(imaging_times, nwb_file):
    for i in imaging_times.index:
        epoch_name = str(i)[0:5]
        data = list(range(100, 200, 10))
        timestamps = list(range(10))
                        
        # # Matlab script loading and running working
        # g_ts, r_ts = eng.pipeline_neuroSEE_batch(tif_path,'/Volumes/Schultz_group_data/Crazy Eights/Ann/GCaMP6 imaging/2018.03.06/2P/20180306_16_57_28_2P/Composite2nd.jpg', nargout=2)
                
        red_ts = TimeSeries(epoch_name + ' ' + 'red_timeseries', #this names the TS in the epoch 
                                     'PyNWB tutorial', #THIS NEEDS TO BE CHANGED
                                     data, 
                                     'SI unit', 
                                     starting_time = 1.0, 
                                     rate = 1.0
                           )
        green_ts = TimeSeries(epoch_name + ' ' + 'green_timeseries', #this names the TS in the epoch 
                                     'PyNWB tutorial', #THIS NEEDS TO BE CHANGED
                                     data, 
                                     'SI unit', 
                                     starting_time = 0.0, 
                                     rate = 1.0
                             )
        
        nwb_file.create_epoch(epoch_name,' ', 2.0, 4.0, [1, 2], imaging_times.loc[i, 'Imaging details'] + '. ' + imaging_times.loc[i, 'Remarks'])
        
        nwb_file.set_epoch_timeseries(epoch_name , red_ts)
#         nwb_file.set_epoch_timeseries(epoch_name , green_ts)
        
def behav_TS(behav_img_session, nwb_file):
    for behaviour in behav_img_session.index:
        epoch_name = str(behaviour)[0:5]
        data = list(range(100, 200, 10))
    
        behav_ts = TimeSeries(epoch_name + ' behaviour',
                             'PyNWB tutorial', #THIS NEEDS TO BE CHANGED
                             data,
                             'SI Unit',
                             starting_time = 0.0,
                             rate = 1.0
                             )
    
        nwb_file.set_epoch_timeseries(epoch_name , behav_ts)
    
# returns the folder of the 2p data for analysis
# need to match each time from exp sheet with folder of 2p data
def corresponding_2P(data_sheet, i, imaging_folder, exp_date, imaging_times):
    
    twop_folders = os.listdir(imaging_folder)
    twop_folders = sorted(twop_folders)
    
    if len(twop_folders) != len(imaging_times.index):
         print("Spreadsheet doesn't match the imaging folders for" + imaging_folder)
    else:
        for folder, img_time in zip(twop_folders, imaging_times.index):
            folder_time, sheet_time = folder[9:14].replace('_',':'), str(img_time)[0:5]
            if folder_time != sheet_time:
                wb = load_workbook(data_sheet, read_only=False, keep_vba=True)
                sheet = wb[i]
                for incorrect_time in sheet['A']:
                    if sheet_time == str(incorrect_time.value)[0:5]:
                        sheet['A' + str(incorrect_time.row)] = sheet_time
                        wb.save(data_sheet[:-4]+'xlsm')

In [8]:
def get_data(data_sheet, imaging_folder, behav_folder):
    sheets = sbj_sheet_list(data_sheet)
    # using the square brackets you can construct paths easier (look into this)
    sbj_details = [x for x in sheets if is_sbj_sheet(x)]
    for i in sbj_details:
        sbj = pd.read_excel(data_sheet, sheet_name= i , index_col=0)
        times = sbj.loc['Time':].copy()
        times.columns = times.iloc[0]
        times.drop(['Time'],inplace=True)
        
#        This returns a check on whether there is a TRUE inside the column
        imaging_times = times[times['2P (X)'].notnull()]
        behav_img_session = imaging_times[imaging_times['IR (X)'].notnull()]
        print(type(behav_img_session.index))
        
        if len(imaging_times) > 0:
            
            mouse_id = sbj.loc['Number','Unnamed: 1']
            exp_data = get_exp_data(data_sheet)
            exp_date = exp_data.at['Date','Unnamed: 1'].replace('.', '')
            mouse_date_folder = create_mouse_date_folder(mouse_id, exp_date)
            
            # Looking for existing NWB file for this session
            session_start = str(imaging_times.index[0])[0:5]
            output_file_name = mouse_id + '_' + exp_date + '_' +  session_start.replace(':','_') + '.nwb'
            output_filepath = os.path.join(mouse_date_folder, output_file_name)

            if not os.path.exists(output_filepath):
                nwb_file = NWBFile(
                            source = '',
                            session_description = mouse_id + ' ' + exp_date + ' ' + session_start,
                            identifier = mouse_id + ' ' + exp_date + ' ' + session_start, 
                            session_start_time = session_start,
                #            file_create_date = datetime.now(tz=None),
                #            experimenter='Dr Ann Go',
                            lab = 'Neural Coding',
                            institution = 'Imperial College London',
                            experiment_description = 'GCamp Imaging',
                            virus = sbj.index[6] + ':' + str(sbj.loc['GCaMP Inj','Unnamed: 1']) + ', ' \
                            + sbj.index[7] + ':' + str(sbj.loc['Methoxy Inj','Unnamed: 1'])
                            )  
                
                #find folder coressponding to 2P data
                # TRYING TO WRITE CORRECT TIMES TO SPREADSHEET BUT WRITTEN XLSM FILE IS THEN CORRUPT
#                 corresponding_2P(data_sheet, i, imaging_folder,exp_date,imaging_times)
            
# #               Build the path to the .tif  
#                 if len(twop_path) == 1:
#                     if twop_path[0] + '_XYT.tif' in os.listdir(twop_path[0]):
#                         tif_path = twop_path[0] + '.tif'
#                         # test data
                        
#                     else:
#                         print('No .tif file')
#                 else:
#                     print('Multiple imaging folders for ' + twop_path)

                twop_TS(imaging_times, nwb_file)
                behav_TS(behav_img_session, nwb_file)
        
#               Build path to behaviour file
#               If there is a corresponding X in the behaviour column, then we look for file
                # This builds a file name, but you can also get a list of all available ones and link it back?
                # which one is better? Getting a ist would allow for name changes but then how would it link back
                behav_file_names = [mouse_id + '_' + x[1] + '.mp4' for x in behav_img_session['IR (X)']]
                behav_file_paths = [os.path.join(behav_folder,x) for x in behav_file_names]
                
                file = NWBHDF5IO(output_filepath, manager=get_manager(), mode='w')
                file.write(nwb_file)
                file.close()
#             else:
#                 io = NWBHDF5IO(output_filepath)
#                 nwbfile = io.read()
#                 find_epoch = nwbfile.get_epochs()
#                 print(find_epoch)

In [9]:
get_data('/Volumes/Schultz_group_data/Crazy Eights/Ann/GCaMP6 imaging/2018.03.02/Experiment Data Sheet - 2018.03.02.xlsm','/Volumes/Schultz_group_data/Crazy Eights/Ann/GCaMP6 imaging/2018.03.02/2P/','/Volumes/Schultz_group_data/Crazy Eights/Ann/GCaMP6 imaging/2018.03.02/IR tracking')

<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
